# Science Performance Metrics - Photometric
<br>Owner(s): **Leanne Guy** ([@leannep](https://github.com/?body=@leannep))
<br>Last Verified to Run: **2021-02-19**
<br>Verified Stack Release: **21.0.0**

This notebook demonstrates the [Photometric Science Performance Metrics](https://ls.st/dmsr) 

### Learning Objectives:

After working through this tutorial you should be able to: 
1. Run the Photometric Science Performance Metrics
2. ....

### Logistics
This notebook is intended to be runnable on `data.lsst.cloud` from a local git clone of https://github.com/leannep/.

## Set-up

In [2]:
# What version of the Stack are we using?
! echo $HOSTNAME
! eups list -s | grep faro

nb-leannep-d-2021-03-09
faro                  master-g65043810f4+1caa810786 	d_2021_03_09 current setup


In [2]:
%pwd

'/home/leannep/notebooks'

In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from itertools import cycle
from astropy.io import fits
import time,glob,os

import lsst.pipe.base as pipeBase

#import warnings
#warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

In [16]:
# What version of the Stack am I using?
! echo $HOSTNAME
! eups list -s lsst_distrib

# Make a directory 
username=os.environ.get('USERNAME')
base_dir='/home/'+username+'/DATA/' +'srd_metrics'
if not os.path.exists(base_dir):
    print("Data directory does not exist: " + base_dir)
    print("Create and setup test data before continuing")
    
testdata_dir = os.path.join(base_dir, "test_data")
if not os.path.exists(base_dir):
    print("Test data dir: "+ testdata_dir + "not found ... setup before continuing")
print("Test data in: " + testdata_dir)

nb-leannep-w-2021-04
   21.0.0-1-g00ce914+8f964e8600 	w_2021_04 current setup
Test data in: /home/leannep/DATA/srd_metrics/test_data


In [30]:
# Create a matched catalog
from lsst.afw.table import SimpleCatalog
PA1_i_cat = "matchedCatalogTract_0_i.fits.gz"
PA1_r_cat = "matchedCatalogTract_0_r.fits.gz"
catalog_pa1_i = SimpleCatalog.readFits(os.path.join(testdata_dir, PA1_i_cat))
catalog_pa1_r = SimpleCatalog.readFits(os.path.join(testdata_dir, PA1_r_cat))

In [31]:
# Expected  PA1 results
import yaml
expected_pa1_i = yaml.load(os.path.join(testdata_dir, 'PA1_expected_0_i.yaml'), Loader=yaml.FullLoader)
expected_pa1_r = yaml.load(os.path.join(testdata_dir, 'PA1_expected_0_r.yaml'), Loader=yaml.FullLoader)

In [6]:
filter_dict = {'u': 1, 'g': 2, 'r': 3, 'i': 4, 'z': 5, 'y': 6,
               'HSC-U': 1, 'HSC-G': 2, 'HSC-R': 3, 'HSC-I': 4, 'HSC-Z': 5, 'HSC-Y': 6}

In [112]:
from lsst.afw.table import GroupView
def filterMatches(matchedCatalog, snrMin=None, snrMax=None,
                  extended=None, doFlags=None, isPrimary=None,
                  psfStars=None, photoCalibStars=None,
                  astromCalibStars=None):

    if snrMin is None:
        snrMin = 50.0
    if snrMax is None:
        snrMax = np.Inf
    if extended is None:
        extended = False
    if doFlags is None:
        doFlags = True
    nMatchesRequired = 2
    if isPrimary is None:
        isPrimary = True
    if psfStars is None:
        psfStars = False
    if photoCalibStars is None:
        photoCalibStars = False
    if astromCalibStars is None:
        astromCalibStars = False

    matchedCat = GroupView.build(matchedCatalog)
    magKey = matchedCat.schema.find('slot_PsfFlux_mag').key

    def nMatchFilter(cat):
        if len(cat) < nMatchesRequired:
            return False
        return np.isfinite(cat.get(magKey)).all()

    def snrFilter(cat):
        # Note that this also implicitly checks for psfSnr being non-nan.
        snr = cat.get('base_PsfFlux_snr')
        ok0, = np.where(np.isfinite(snr))
        medianSnr = np.median(snr[ok0])
        return snrMin <= medianSnr and medianSnr <= snrMax

    def ptsrcFilter(cat):
        ext = cat.get('base_ClassificationExtendedness_value')
        # Keep only objects that are flagged as "not extended" in *ALL* visits,
        # (base_ClassificationExtendedness_value = 1 for extended, 0 for point-like)
        if extended:
            return np.min(ext) > 0.9
        else:
            return np.max(ext) < 0.9

    def flagFilter(cat):
        if doFlags:
            flag_sat = cat.get("base_PixelFlags_flag_saturated")
            flag_cr = cat.get("base_PixelFlags_flag_cr")
            flag_bad = cat.get("base_PixelFlags_flag_bad")
            flag_edge = cat.get("base_PixelFlags_flag_edge")
            return np.logical_not(np.any([flag_sat, flag_cr, flag_bad, flag_edge]))
        else:
            return True

    def isPrimaryFilter(cat):
        if isPrimary:
            flag_isPrimary = cat.get("detect_isPrimary")
            return np.all(flag_isPrimary)
        else:
            return True

    def fullFilter(cat):
        return nMatchFilter(cat) and snrFilter(cat) and ptsrcFilter(cat)\
            and flagFilter(cat) and isPrimaryFilter(cat)

    return matchedCat.where(fullFilter)

In [111]:
def computeWidths(array):
    # For scalars, math.sqrt is several times faster than numpy.sqrt.
    rmsSigma = math.sqrt(np.mean(array**2))
    iqrSigma = np.subtract.reduce(np.percentile(array, [75, 25])) / (norm.ppf(0.75)*2)
    return rmsSigma, iqrSigma

In [110]:
def getRandomDiffRmsInMmags(array, rng=None):
    thousandDivSqrtTwo = 1000/math.sqrt(2)
    return thousandDivSqrtTwo * getRandomDiff(array, rng=rng)

In [109]:
def getRandomDiff(array, rng=None):
    if not rng:
        rng = np.random.default_rng()
    a, b = rng.choice(range(len(array)), 2, replace=False)
    return array[a] - array[b]

In [108]:
import functools
import math
import numpy as np
from scipy.stats import norm
import astropy.units as u

def calcPhotRepeatSample(matches, magKey, rng=None):
    sampler = functools.partial(getRandomDiffRmsInMmags, rng=rng)
    magDiffs = matches.aggregate(sampler, field=magKey)
    magMean = matches.aggregate(np.mean, field=magKey)
    rms, iqr = computeWidths(magDiffs)
    return pipeBase.Struct(rms=rms, iqr=iqr, magDiffs=magDiffs, magMean=magMean,)

In [148]:
# Calculate the photometric repeatability of measurements across a set of randomly selected pairs of visits.
def calcPhotRepeat(matches, magKey, numRandomShuffles=50, randomSeed=None):
    rng = np.random.default_rng(randomSeed)
    
    # Selects the random pairs of visits 
    mprSamples = [calcPhotRepeatSample(matches, magKey, rng=rng)
                  for _ in range(numRandomShuffles)]

    for mpr in mprSamples:
        r = mpr.rms *  u.mmag
        i = mpr.iqr * u.mmag
        print(str(r) + ', '  + str(i))
        
    
    rms = np.array([mpr.rms for mpr in mprSamples]) * u.mmag
    iqr = np.array([mpr.iqr for mpr in mprSamples]) * u.mmag
    magDiff = np.array([mpr.magDiffs for mpr in mprSamples]) * u.mmag
    magMean = np.array([mpr.magMean for mpr in mprSamples]) * u.mag
    repeat = np.mean(iqr)
    return {'rms': rms, 'iqr': iqr, 'magDiff': magDiff, 'magMean': magMean, 'repeatability': repeat}

In [141]:
# Compute photometric repeatability 
def photRepeat(matchedCatalog, numRandomShuffles=50, randomSeed=None, **filterargs):
    
    # Apply selections for bright, isolated point sources on the matched catalog
    filteredCat = filterMatches(matchedCatalog, **filterargs)
    magKey = filteredCat.schema.find('slot_PsfFlux_mag').key

    # Require at least nMinPhotRepeat objects to calculate the repeatability:
    nMinPhotRepeat = 50
    if filteredCat.count > nMinPhotRepeat:
        phot_resid_meas = calcPhotRepeat(filteredCat, magKey, numRandomShuffles=50, randomSeed=randomSeed)
        return phot_resid_meas
    else:
        return {'nomeas': np.nan*u.mmag}

In [149]:
# PA1 
metric_name = "PA1"
band = 'i'

# Configuration 
"Minimum median SNR for a source to be considered bright."
brightSnrMin = 50

"Maximum median SNR for a source to be considered bright."
brightSnrMax = np.inf

"Number of trials used for random sampling of observation pairs."
numRandomShuffles = 50

"Random seed for sampling."
randomSeed = 12345

print("Measuring "+ metric_name + " band: " + band)
pa1 = photRepeat(catalog_pa1_i, snrMax=brightSnrMax, snrMin=brightSnrMin, 
                 numRandomShuffles=numRandomShuffles, randomSeed=randomSeed)

if not 'magDiff' in pa1.keys():
    print("Mag diff nor computed")
else:
    assert pa1['repeatability'].value == 13.820912720667566, "Wrong value"
    print("pa1 repeatability: " + str(pa1['repeatability']))



Measuring PA1 band: i
15.003075050688878 mmag, 13.803541490795373 mmag
14.771605495091457 mmag, 14.31340202812624 mmag
15.01236863651567 mmag, 14.071282270638527 mmag
15.128323595384904 mmag, 14.423013274569831 mmag
14.7208018726209 mmag, 13.888378689880234 mmag
14.909305426203971 mmag, 14.158208280518075 mmag
15.173045788205352 mmag, 13.925115870870316 mmag
15.15646161002913 mmag, 13.481592115485112 mmag
14.722617129488759 mmag, 13.305003772354185 mmag
14.963398573561754 mmag, 14.024765026422019 mmag
14.7207074250828 mmag, 14.05206011061424 mmag
14.788682347523363 mmag, 13.934184192305098 mmag
14.749725011477306 mmag, 14.377388962488931 mmag
14.81971687998953 mmag, 13.509716272530884 mmag
14.57650353857061 mmag, 13.53802249323352 mmag
14.585646279700303 mmag, 13.720690559322072 mmag
14.50116270430295 mmag, 13.53615789301272 mmag
14.75255980046964 mmag, 13.463397039033024 mmag
14.532269977526939 mmag, 13.201161085655798 mmag
14.537538977142539 mmag, 13.435042178559888 mmag
14.656213960

In [ ]:
# PA1 expected - PA1_expected_0_i.yaml, PA1_expected_0_r.yaml

# AD / AF

In [162]:
sepDistances = np.array([20, 17,19,21,12,10,7,3,13,16])
absDiffsMarcsec = np.abs((sepDistances-np.median(sepDistances))) * u.marcsec
print(absDiffsMarcsec)
afPercentile = 0.1
pctl = np.percentile(absDiffsMarcsec.value, afPercentile)*u.marcsec
print(pctl)

[ 5.5  2.5  4.5  6.5  2.5  4.5  7.5 11.5  1.5  1.5] marcsec
1.5 marcsec
